In [1]:
import tensorflow as tf

tf.__version__

'2.0.0'

In [2]:
def load_train_valid_data(valid_ratio=0.2, show_info=False):
    import mnist
    from sklearn.model_selection import train_test_split

    images, labels = mnist.train_images(), mnist.train_labels()
    x_train, x_valid, y_train, y_valid = train_test_split(
            images, labels, test_size=valid_ratio, random_state=111)

    if show_info:
        print("Train images:", type(x_train), x_train.shape, x_train.dtype)
        print("Train labels:", type(y_train), y_train.shape, y_train.dtype)
        print("Valid images:", type(x_valid), x_valid.shape, x_valid.dtype)
        print("Valid labels:", type(y_valid), y_valid.shape, y_valid.dtype)

    return (x_train, y_train), (x_valid, y_valid)


def load_test_data():
    import mnist

    return mnist.test_images(), mnist.test_labels()


def preprocess(x, y):
    x = x.reshape(-1, 784).astype("float32")/255.
    return x, y

In [3]:
## Hyper parameters
n_epochs = 10
batch_size = 256


## Load dataset
(x_train, y_train), (x_valid, y_valid) = load_train_valid_data()
x_train, y_train = preprocess(x_train, y_train)
x_valid, y_valid = preprocess(x_valid, y_valid)

print("Train images:", type(x_train), x_train.shape, x_train.dtype)
print("Train labels:", type(y_train), y_train.shape, y_train.dtype)
print("Valid images:", type(x_valid), x_valid.shape, x_valid.dtype)
print("Valid labels:", type(y_valid), y_valid.shape, y_valid.dtype)


## Model / Loss function / Optimizer
input_size = x_train.shape[0]
output_size = int(y_train.max()) + 1

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optim = tf.keras.optimizers.Adam()

Train images: <class 'numpy.ndarray'> (48000, 784) float32
Train labels: <class 'numpy.ndarray'> (48000,) uint8
Valid images: <class 'numpy.ndarray'> (12000, 784) float32
Valid labels: <class 'numpy.ndarray'> (12000,) uint8


In [4]:
model.compile(loss=loss_fn, optimizer=optim)
model.fit(x_train, y_train, validation_data=(x_valid, y_valid),
    epochs=n_epochs, batch_size=batch_size)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 2s 45us/sample - loss: 0.3906 - val_loss: 0.1940
Epoch 2/10
48000/48000 [==============================] - 2s 32us/sample - loss: 0.1507 - val_loss: 0.1327
Epoch 3/10
48000/48000 [==============================] - 2s 32us/sample - loss: 0.1026 - val_loss: 0.1094
Epoch 4/10
48000/48000 [==============================] - 1s 31us/sample - loss: 0.0772 - val_loss: 0.1035
Epoch 5/10
48000/48000 [==============================] - 1s 31us/sample - loss: 0.0590 - val_loss: 0.0872
Epoch 6/10
48000/48000 [==============================] - 2s 31us/sample - loss: 0.0434 - val_loss: 0.0897
Epoch 7/10
48000/48000 [==============================] - 2s 32us/sample - loss: 0.0347 - val_loss: 0.0843
Epoch 8/10
48000/48000 [==============================] - 2s 32us/sample - loss: 0.0259 - val_loss: 0.0902
Epoch 9/10
48000/48000 [==============================] - 2s 32us/sample - loss: 0.0201 - val_

In [6]:
train_data = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(buffer_size=10000).batch(batch_size)

valid_data = tf.data.Dataset.from_tensor_slices(
    (x_valid, y_valid)).batch(batch_size)

print("\n[Training with validation]")
history = {'train_loss':[], 'valid_loss':[]}
for epoch in range(n_epochs):

    ## Training
    batch_loss = []
    for xi, yi in train_data:
        with tf.GradientTape() as tape:
            yi_hat = model(xi, training=True)
            loss = loss_fn(yi, yi_hat)

        grads = tape.gradient(loss, model.trainable_weights)
        optim.apply_gradients(zip(grads, model.trainable_weights))
        batch_loss.append(float(loss))

    train_loss = sum(batch_loss)/len(batch_loss)

    ## Validation
    batch_loss = []
    for xi, yi in valid_data:
        yi_hat = model(xi, training=False)
        loss = loss_fn(yi, yi_hat)
        batch_loss.append(float(loss))

    valid_loss = sum(batch_loss)/len(batch_loss)

    ## Print log
    print("Epoch [%3d/%3d] >>> train_loss = %.2e, valid_loss = %.2e" 
        % (epoch+1, n_epochs, train_loss, valid_loss))

    history['train_loss'].append(train_loss)
    history['valid_loss'].append(valid_loss)


[Training with validation]
Epoch [  1/ 10] >>> train_loss = 1.44e-02, valid_loss = 8.55e-02
Epoch [  2/ 10] >>> train_loss = 1.03e-02, valid_loss = 8.44e-02
Epoch [  3/ 10] >>> train_loss = 8.21e-03, valid_loss = 8.57e-02
Epoch [  4/ 10] >>> train_loss = 8.50e-03, valid_loss = 8.85e-02
Epoch [  5/ 10] >>> train_loss = 4.77e-03, valid_loss = 9.23e-02
Epoch [  6/ 10] >>> train_loss = 5.06e-03, valid_loss = 1.01e-01
Epoch [  7/ 10] >>> train_loss = 4.39e-03, valid_loss = 9.20e-02
Epoch [  8/ 10] >>> train_loss = 2.84e-03, valid_loss = 9.10e-02
Epoch [  9/ 10] >>> train_loss = 1.26e-03, valid_loss = 8.94e-02
Epoch [ 10/ 10] >>> train_loss = 6.46e-04, valid_loss = 9.32e-02


In [61]:
batch_size = 4
data = tf.range(10)*10

data_size = data.shape[0] # 10
steps_per_epoch = data_size // batch_size + (1 if data_size % batch_size else 0) # 3

indices = tf.random.shuffle(tf.range(data.shape[0]))
x = tf.gather(data, indices=indices)

[x[i*batch_size:(i+1)*batch_size] for i in range(steps_per_epoch)]


# [batch_size for _ in range(data.shape[0] // batch_size)]

[<tf.Tensor: id=245676, shape=(4,), dtype=int32, numpy=array([90, 30, 60,  0], dtype=int32)>,
 <tf.Tensor: id=245680, shape=(4,), dtype=int32, numpy=array([80, 10, 50, 70], dtype=int32)>,
 <tf.Tensor: id=245684, shape=(2,), dtype=int32, numpy=array([40, 20], dtype=int32)>]